# 1. 读写 CSV 数据

将数据读取为元组序列

```python
import csv

with open('stocks.csv') as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    
    for row in f_csv:
        # process row
        ...
```

元组命名，只在每一列的标头都是合法的 Python 标识符时才起作用

```python
from collections import namedtuple

with open('stock.csv') as f:
    f_csv = csv.reader(f)
    headings = next(f_csv)
    Row = namedtuple('Row', headings)
    for r in f_csv:
        row = Row(*r)
        # process row
        ...
```

将数据读取为字典序列

```python
import csv

with open('stocks.csv') as f:
    f_csv = csv.DictReader(f)
    for row in f_csv:
        # process row
        ...
```

写入对象

```python
with open('stocks.csv', 'w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)
```

如果是字典

```python
with open('stocks.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(rows)
```

csv 库能识别微软 Excel 所采用的 CSV 编码规则，但可以有几种方法将编码调整为其他的格式

如果想要读取以 tab 键分隔的数据

```python
with open('stock.tsv') as f:
    f_tsv = csv.reader(f, delimiter='\t')
    for row in f_tsv:
        # process row
        ...
```

如果正在读取 CSV 数据并将其转换为命名数组，其中某个 CSV 文件中标题行中包含有非法的标识符字符，会使得创建命名元组的代码出现异常。要解决这个问题，应该首先整理标题，例如，对非法的标识符字符进行正则替换

```python
import re

with open('stock.csv') as f:
    f_csv = csv.reader(f)
    headers = [re.sub('[a-zA-z_]', '_', h) for h in next(f_csv)]
    Row = namedtuple('Row', headers)
    for r in f_csv:
        row = Row(*r)
        # process row
        ...
```

csv 不会解释数据或将数据转换为除字符串外的类型。需要自行转换

```python
col_types = [str, float, str, str, float, int]
with open('stocks.csv') as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    for row in f_csv:
        row = tuple(convert(value) for convert, value in zip(col_types, row))
        ...
```

将选中的字段转换为字典

```python
print('Reading as dicts with type conversion')
field_types = [('Price', float), ('Change', float), ('Volume', int)]

with open('stock.csv') as f:
    for row in csv.DictReader(f):
        row.update((key, conversion(row[key])) for key, conversion in field_types)
        print(row)
```

# 2. 读取 JSON (JavaScript Object Notation) 格式的数据

## 2.1 Python 与 JSON 数据结构转换

将 Python 数据结构转换为 JSON

In [3]:
import json

data = {'name': 'ACME', 'shares': 100, 'price': 542.23}
json_str = json.dumps(data)
json_str

'{"name": "ACME", "shares": 100, "price": 542.23}'

将 JSON 字符串转换为 Python 数据结构

In [4]:
data = json.loads(json_str)
data

{'name': 'ACME', 'shares': 100, 'price': 542.23}

## 2.2 文件编码和解码

对于文件，可以使用 json.dump() 和 json.load() 编码和解码

```python
with open('data.json', 'w') as f:
    json.dump(data, f)
    
with open('data.json', 'r') as f:
    data = json.load(f)
```

JSON 支持 None, bool, int, float, str, 以及列表、元组、字典。对于字典，JSON 会假设键（key）是字符串（字典中的任何非字符串键都会在编码时转换为字符串）。JSON 与 Python 相比，语法几乎一直，但会将 True 映射为 true，False 映射为 false，None 被映射为 null。

要检查从 JSON 中解码得到的数据，仅将其打印出来确定数据的结构是比较困难的，需要用到 pprint 的 pprint()，把键按照字母顺序排列，并将字典以更加合理的方式输出

```python
from urllib.request import urlopen
import json
from pprint import pprint

u = urlopen('http://search.twitter.com/search.json?q=python&rpp=5')
resp = json.loads(u.read().decode('utf-8'))

pprint(resp)
```

JSON 解码时会从所提供的数据中创建出字典或列表，如果想创建其他类的对象，可以为 json.loads() 方法提供 object_pairs_hook 或 object_hook 参数。例如将 JSON 数据解码为 OrderedDict（有序词典），保持数据顺序不变

In [10]:
s = '{"name": "ACME", "shares": 50, "price": 490.1}'
from collections import OrderedDict
data = json.loads(s, object_pairs_hook=OrderedDict)
data

OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])

将 JSON 字典转变为 Python 对象

通过解码 JSON 数据而创建的字典作为单独的参数传递给了 ``__init__()``

In [12]:
class JSONObject:
    def __init__(self, d):
        self.__dict__  = d
        
data = json.loads(s, object_hook=JSONObject)

print(data.name)
print(data.shares)
print(data.price)

ACME
50
490.1


如果在输出中对键进行排序处理，可以使用 sort_keys 参数

如果想让输出的格式变得漂亮一些，可以使用 json.dumps() 中的 indent 参数

In [16]:
data = json.loads(s, object_pairs_hook=OrderedDict)

In [17]:
print(json.dumps(data))

{"name": "ACME", "shares": 50, "price": 490.1}


In [19]:
print(json.dumps(data, indent=4))

{
    "name": "ACME",
    "shares": 50,
    "price": 490.1
}


想在输出中进行排序处理

In [18]:
print(json.dumps(data, sort_keys=True))

{"name": "ACME", "price": 490.1, "shares": 50}


## 2.3 类实例

类实例一般无法序列化为 JSON，如果想序列化类实例，可以提供一个函数将类实例作为输入并返回一个可以被序列化的字典

```python
def serialize_instance(obj):
    d = {'__classname__': type(obj).__name__}
    d.update(vars(obj))
    return d
```

如果想取回一个实例

```python
# 字典映射名为 known classes
classes = {
    'Point': Point
}

def unserialize_object(d):
    clsname = d.pop('__classname__', None)
    if clsname:
        cls = classes[clsname]
        obj = cls.__new__(cls)
        for key, value in d.items():
            setattr(obj, key, value)
            return obj
    else:
        return d
```

# 3. 解析简单的 XML(Extensible Markup Language，可扩展标记语言) 文档

从一个 XML 文档中提取数据

假设对 Planet Python (http://planet.python.org) 上的 RSS 订阅做解析并生成一个总结报告

In [4]:
from urllib.request import urlopen
from xml.etree.ElementTree import parse

u = urlopen('http://planet.python.org/rss20.xml')
doc = parse(u)

for item in doc.iterfind('channel/item'):
    title = item.findtext('title')
    date = item.findtext('pubDate')
    link = item.findtext('link')
    
    print(title)
    print(date)
    print(link)
    print()

Stack Abuse: The 'u' and 'r' String Prefixes and Raw String Literals in Python
Sat, 09 Sep 2023 16:05:00 +0000
https://stackabuse.com/the-u-and-r-string-prefixes-and-raw-string-literals-in-python/

Stack Abuse: When to Use Shebangs in Python Scripts
Sat, 09 Sep 2023 14:12:00 +0000
https://stackabuse.com/when-to-use-shebangs-in-python-scripts/

Stack Abuse: Importing Multiple CSV Files into a Single DataFrame using Pandas in Python
Fri, 08 Sep 2023 21:09:59 +0000
https://stackabuse.com/importing-multiple-csv-files-into-a-single-dataframe-using-pandas-in-python/

Stack Abuse: Determining the Size of an Object in Python
Fri, 08 Sep 2023 17:21:44 +0000
https://stackabuse.com/determining-the-size-of-an-object-in-python/

Python Engineering at Microsoft: Python in Visual Studio Code – September 2023 Release
Fri, 08 Sep 2023 15:43:10 +0000
https://devblogs.microsoft.com/python/python-in-visual-studio-code-september-2023-release/

Mike Driscoll: How to Validate an IP Address in Python
Fri, 08 

在许多情况下，XML 如果只是简单地用来保存数据，那么文档的结构是紧凑而直接的。xml.etree.ElementTree.parse() 将整个 XML 文档解析成一个文档对象。之后就可以使用 find(), iterfind(), findtext() 方法查询特定的 XML 元素。

对于更加高级的应用，可以使用 lxml。lxml 运行起来迅速，还提供验证、XSLT、XPath 等功能。

# 4. 以增量的方式解析大型 XML 文件

从一个大型的 HTML 文档中提取数据，而且对内存的使用尽可能少，应该考虑使用迭代器和生成器

```python
from xml.etree.ElementTree import iterparse

def parse_and_remove(filename, path):
    path_parts = path.split('/')
    doc = iterparse(filename, ('start', 'end'))
    next(doc)
    
    tag_stack = []
    elem_stack = []
    for event, elem in doc:
        if event == 'start':
            tag_stack.append(elem.tag)
            elem_stack.append(elem)
        elif event == 'end':
            if tag_stack == path_parts:
                yield elem
                elem_stack[-2].remove(elem)
            try:
                tag_stack.pop()
                elem_stack.pop()
            except IndexError:
                pass
```

编写一个脚本根据坑洞的数量对邮政编码 (ZIP code) 进行排序

```python
from xml.etree.ElementTree import parse
from collections import Counter

potholes_by_zip = Counter()

doc = parse('potholes.xml')
for pothole in doc.iterfind('row/row'):
    potholes_by_zip[pothole.findtext('zip')] += 1
for zipcode, num in potholes_by_zip.most_common():
    print(zipcode, num)
```

这个脚本将整个 XML 读取到内存后再解析，运行这个脚本要占用 450 MB 内存

```python
from collections import Counter
potholes_by_zip = Counter()

data = parse_and_remove('potholes.xml', 'row/row')
for pothole in data:
    potholes_by_zip[pothole.findtext('zip')] += 1

for zipcode, num in potholes_by_zip.most_common():
    print(zipcode, num)
```

运行这个代码只需要 7 MB 内存

iterparse()允许对 XML 文档做增量式处理，只需要提供文件名和一个事件列表。事件列表由一个或多个 start/end, start-ns/end-ns 组成。iterparse() 创建出的迭代器产出形式为 (event, elem) 的元组，event 是列出的事件，elem 是对应的 XML 元素

当某个元素首次被创建但还没有填入其他数据时，会产生 start 事件，而 end 事件会在元素已经完成时产生。start-ns, end-ns 事件是用来处理 XML 命名空间声明的